In [1]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("manjilkarki/deepfake-and-real-images")

# print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/deepfake-and-real-images


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import classification_report
from tqdm import tqdm
import kagglehub

In [3]:
# ---- Device Setup ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 2
BATCH_SIZE = 32
EPOCHS = 5

In [4]:
# ---- Transforms ----
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

In [5]:
# ---- Helper: Fix Class Labels ----
def remap_targets(dataset):
    mapping = {"Real": 0, "Fake": 1}
    dataset.samples = [(p, mapping[p.split("/")[-2]]) for p, _ in dataset.samples]

In [7]:
train_dataset = datasets.ImageFolder(f"/kaggle/input/deepfake-and-real-images/Dataset/Train", transform=transform)
remap_targets(train_dataset)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [9]:
val_dataset = datasets.ImageFolder(f"/kaggle/input/deepfake-and-real-images/Dataset/Validation", transform=transform)
remap_targets(val_dataset)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [10]:
test_dataset = datasets.ImageFolder(f"/kaggle/input/deepfake-and-real-images/Dataset/Test", transform=transform)
remap_targets(test_dataset)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [11]:
# ---- Hybrid Model ----
class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.patch_embed = nn.Conv2d(256, 128, kernel_size=4, stride=4)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=128, nhead=4, dim_feedforward=256),
            num_layers=2
        )
        self.classifier = nn.Linear(128, NUM_CLASSES)

    def forward(self, x):
        x = self.cnn(x)
        x = self.patch_embed(x)         # [B, 128, 8, 8]
        x = x.flatten(2).permute(2, 0, 1)  # [64, B, 128]
        x = self.transformer(x)         # [64, B, 128]
        x = x.mean(dim=0)               # [B, 128]
        return self.classifier(x)


In [12]:
# ---- Train & Eval Functions ----
def train(model, loader, optimizer, criterion):
    model.train()
    running_loss, correct = 0, 0
    for x, y in tqdm(loader, desc="Training"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        correct += (out.argmax(1) == y).sum().item()
    acc = correct / len(loader.dataset)
    return running_loss / len(loader), acc

def evaluate(model, loader, criterion):
    model.eval()
    loss, correct = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss += criterion(out, y).item()
            correct += (out.argmax(1) == y).sum().item()
    acc = correct / len(loader.dataset)
    return loss / len(loader), acc

In [13]:
# ---- Training Loop ----
model = HybridModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_loader, criterion)
    print(f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(



Epoch 1/5


Training: 100%|██████████| 4376/4376 [18:48<00:00,  3.88it/s]


Train Acc: 0.8241 | Val Acc: 0.8794

Epoch 2/5


Training: 100%|██████████| 4376/4376 [06:54<00:00, 10.57it/s]


...

Train Acc: 0.9442 | Val Acc: 0.9258

Epoch 4/5


Training: 100%|██████████| 4376/4376 [07:03<00:00, 10.32it/s]


Train Acc: 0.9547 | Val Acc: 0.9295

Epoch 5/5


Training: 100%|██████████| 4376/4376 [07:17<00:00,  9.99it/s]


Train Acc: 0.9628 | Val Acc: 0.9209


In [14]:
# ---- Testing ----
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for x, y in tqdm(test_loader, desc="Testing"):
        x = x.to(device)
        out = model(x)
        preds = out.argmax(1).cpu()
        y_true.extend(y.numpy())
        y_pred.extend(preds.numpy())

print("\nTest Classification Report:")
print(classification_report(y_true, y_pred, target_names=["REAL", "FAKE"]))

Testing: 100%|██████████| 341/341 [01:43<00:00,  3.30it/s]


Test Classification Report:
              precision    recall  f1-score   support

        REAL       0.86      0.95      0.90      5413
        FAKE       0.94      0.85      0.89      5492

    accuracy                           0.90     10905
   macro avg       0.90      0.90      0.90     10905
weighted avg       0.90      0.90      0.90     10905



In [15]:
# ---- Save Trained Model ----
torch.save(model.state_dict(), "hybrid_deepfake_DFRI_dataset.pth")
